In [3]:
import re
import httpx
import os
from dotenv import load_dotenv
import google.generativeai as genai
_ = load_dotenv()

d:\proj\agents\Agents-LangGraph\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)

That's wonderful! Two dogs means double the love and fun. Are they both the same breed? Do they get along well? I'd love to hear more about them. 😊 

You have two dogs, and each dog has four paws.  So there are a total of **eight** paws in your house! 🐾🐾🐾🐾🐾🐾🐾🐾 



In [6]:
model = genai.GenerativeModel("gemini-1.5-flash")

chat_completion = model.start_chat(
    history=[{"role": "user", "parts": "Hello world"}]
)

In [11]:
response = chat_completion.send_message("Hello World")
response.text

"Hello World! It's great to hear from you. What can I do for you today? \n"

In [22]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "model", "parts": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "parts": message})
        result = self.execute(message)
        self.messages.append({"role": "model", "parts": result})
        return result

    def execute(self, message):
        model = genai.GenerativeModel("gemini-1.5-flash")
        completion = model.start_chat(
            history=self.messages
        )
        response = completion.send_message(message)
        return response.text
    

In [23]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [24]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [25]:
abot = Agent(prompt)

In [26]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a toy poodle using the available action.
Action: average_dog_weight: Toy Poodle
PAUSE



In [28]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [29]:
next_prompt = "Observation: {}".format(result)

In [30]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs on average. \n'

In [31]:
abot.messages

[{'role': 'model',
  'parts': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'parts': 'How much does a toy p

In [32]:
abot = Agent(prompt)
abot.messages

[{'role': 'model',
  'parts': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'}]

In [33]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to know the individual weights of each breed. I can use average_dog_weight to find those weights. \nAction: average_dog_weight: Border Collie\nPAUSE \n'

In [34]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [35]:
abot(next_prompt)

'Thought: I now have the Border Collie weight. I need to know the Scottish Terrier weight.\nAction: average_dog_weight: Scottish Terrier\nPAUSE\n'

In [36]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [37]:
abot(next_prompt)

'Thought: I now have the weight of both dogs. I can calculate their combined weight.\nAction: calculate: 37 + 20\nPAUSE\n'

In [38]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [39]:
abot(next_prompt)

'Answer: Your two dogs combined weight is 57 lbs \n'

In [40]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [41]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [42]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of each breed and add them together
Action: average_dog_weight: Border Collie
PAUSE

 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need the average weight of the Scottish terrier
Action: average_dog_weight: Scottish Terrier
PAUSE

 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now I can add those together to get the combined weight
Action: calculate: 37 + 20
PAUSE

 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of your dogs is 57 lbs 

